In [1]:
%matplotlib inline
import sys, os
sys.path.append('../..')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h5py

In [2]:
from sktime.base import load
#from sklearn.model_selection import GridSearchCV
#from sktime.classification.feature_based import TSFreshClassifier
from sktime.classification.deep_learning.cnn import CNNClassifier

In [15]:
period = 50
filepath = os.path.join('..','..','data','trend index','training set', f'trend_{period}day.hdf')
file = h5py.File(filepath, mode='r')

In [20]:
ohlc  = file['ohlc'][:]
trend = file['trend'][:]
# shuffling
idx = np.random.permutation(len(ohlc))
ohlc = ohlc[idx]
trend = trend[idx]

print(ohlc.shape)
print(trend.shape)


(1006, 4, 50)
(1006,)


### 1. Train data & Test data 생성 (7:3)

In [21]:
idx = int(len(ohlc)*0.7)
ohlc_train = ohlc[:idx]
trend_train = trend[:idx]

ohlc_test = ohlc[idx:]
trend_test = trend[idx:]

print(ohlc_train.shape)
print(ohlc_test.shape)

(704, 4, 50)
(302, 4, 50)


### 2. 훈련 모델 생성 및 훈련

In [22]:
network = CNNClassifier(n_epochs=50,batch_size=3, verbose=True)
network.fit(ohlc_train, trend_train)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 50, 4)]           0         
                                                                 
 conv1d_2 (Conv1D)           (None, 50, 6)             174       
                                                                 
 average_pooling1d_2 (Averag  (None, 16, 6)            0         
 ePooling1D)                                                     
                                                                 
 conv1d_3 (Conv1D)           (None, 16, 12)            516       
                                                                 
 average_pooling1d_3 (Averag  (None, 5, 12)            0         
 ePooling1D)                                                     
                                                                 
 flatten_1 (Flatten)         (None, 60)                0   

C:\Users\yosep\python\miniconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


235/235 [==============================] - 1s 878us/step - loss: 0.1602 - accuracy: 0.6406
Epoch 2/50
235/235 [==============================] - 0s 871us/step - loss: 0.1223 - accuracy: 0.7571
Epoch 3/50
235/235 [==============================] - 0s 892us/step - loss: 0.1091 - accuracy: 0.7898
Epoch 4/50
235/235 [==============================] - 0s 944us/step - loss: 0.0981 - accuracy: 0.8068
Epoch 5/50
235/235 [==============================] - 0s 853us/step - loss: 0.0920 - accuracy: 0.8267
Epoch 6/50
235/235 [==============================] - 0s 910us/step - loss: 0.0890 - accuracy: 0.8210
Epoch 7/50
235/235 [==============================] - 0s 859us/step - loss: 0.0834 - accuracy: 0.8352
Epoch 8/50
235/235 [==============================] - 0s 855us/step - loss: 0.0795 - accuracy: 0.8480
Epoch 9/50
235/235 [==============================] - 0s 903us/step - loss: 0.0727 - accuracy: 0.8594
Epoch 10/50
235/235 [==============================] - 0s 855us/step - loss: 0.0711 - accurac

CNNClassifier(batch_size=3, n_epochs=50, verbose=True)

In [23]:
network.score(ohlc_test, trend_test)

101/101 [==============================] - 0s 690us/step


0.9072847682119205

In [24]:
network.predict_proba(ohlc_test)

101/101 [==============================] - 0s 490us/step


array([[2.19735785e-10, 5.88046480e-03, 9.94119525e-01],
       [2.38185366e-05, 4.36188251e-01, 5.63787937e-01],
       [9.96882021e-01, 2.85778637e-03, 2.60198052e-04],
       [4.84409502e-05, 9.98309791e-01, 1.64182740e-03],
       [2.27135642e-05, 9.99776542e-01, 2.00747731e-04],
       [1.27366588e-01, 8.72630417e-01, 2.98463647e-06],
       [1.71277588e-05, 9.99527276e-01, 4.55647998e-04],
       [2.12294108e-05, 9.97237563e-01, 2.74110911e-03],
       [1.18668592e-02, 9.88103926e-01, 2.92602363e-05],
       [5.56432781e-07, 9.94234741e-01, 5.76469162e-03],
       [6.77363038e-01, 3.22529733e-01, 1.07221342e-04],
       [9.86236870e-01, 1.37523906e-02, 1.07234109e-05],
       [2.51579881e-01, 7.48419821e-01, 2.60323446e-07],
       [5.78539661e-10, 1.71746597e-01, 8.28253388e-01],
       [2.16309672e-08, 2.25301627e-02, 9.77469802e-01],
       [1.83879511e-05, 9.96288598e-01, 3.69296595e-03],
       [1.36143963e-05, 9.97385204e-01, 2.60121305e-03],
       [1.10841990e-02, 9.88674

### 4. 검증

In [25]:
from tools.instruments import instruments
from tools.datatools import norm
from tools.display import highchart
quotes = instruments.quotes(fields='ohlc')
symbols = list(quotes.columns.levels[0])

In [26]:
%load_ext autoreload
%aimport tools.datatools
%aimport tools.display
%autoreload 2

In [71]:
symbol = np.random.choice(symbols)
quote = quotes['W'].dropna()
idx = 500#np.random.randint(0,len(quote)-51)
ohlc = quote.iloc[idx:idx+period]
data = norm(ohlc.values)
data = np.moveaxis(np.array([data]),1,2 )
print(network.predict_proba(data).round(2))
highchart([{'data':ohlc, 'type':'candlestick'}])

1/1 [==============================] - 0s 16ms/step
[[0. 1. 0.]]


### 3.모델을 파일로 저장

In [63]:
savepath = os.path.join('..','..','data','trend index','models', f'trend_{period}day')
network.save(savepath)

INFO:tensorflow:Assets written to: ..\..\data\trend index\models\trend_50day\keras\assets


INFO:tensorflow:Assets written to: ..\..\data\trend index\models\trend_50day\keras\assets


<zipfile.ZipFile filename='..\\..\\data\\trend index\\models\\trend_50day.zip' mode='r'>